<a href="https://colab.research.google.com/github/martatolos/eae-dsaa-2025/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Support Vector Machines (SVMs) using Python

Goal of the session: At the end of this tutorial, you will understand the basics of SVM models and how to apply them using `scikit-learn`.

## SVM: Recap theory

**Describe in your own words: what is a support vector machine (SVM) and how does it work?**


<details>
  <summary>💡 Show solution</summary>
  <ul>
    <li>Set of supervised learning methods used for classification (and sometimes for regression and outliers detection).</li>
    <li>Core idea: It finds the best boundary (hyperplane) that separates data points of different classes with the maximum margin.</li>
    <li>Margin: The distance between the hyperplane and the nearest data points of each class. SVM tries to maximize this margin for better generalization.</li>
    <li>Works linearly in basic form but can handle non-linear separation using the kernel trick (e.g., RBF, polynomial kernels) to project data into higher dimensions.</li>
 </ul>
</details>


### Pros and Cons

**Strengths of SVMs:**
- Effective in high-dimensional spaces (i.e., with many features, like text data) - even effective approach even if the number of features is (much) larger than the number of samples
- Memory efficient - uses only a subset of training data (support vectors).
- Non-linearity - can handle non-linear separation using the kernel trick (e.g., RBF, polynomial kernels) to project data into higher dimensions.
- Versatile - can be used with different kernel functions for linear and non-linear classification
- Robust to overfitting - especially in high-dimensional settings with proper regularization

**Weaknesses of SVMs**
- Not suitable for large datasets - training time and memory usage scale poorly with number of samples.
- Performance drops with noisy data and overlapping classes.
- Limited interpretability - unlike decision trees or linear models, SVMs can be hard to explain.
- No direct probability estimates - requires extra calibration if probabilities are needed.
- Hard to tune — choosing the right kernel and hyperparameters (C, gamma) can be tricky.

## SVMs in Python using scikit-learn

### Prerequisite

Before we begin, we have to install the necessary python packages

In [ ]:
%pip install numpy==2.0.2 pandas==2.2.2 scikit-learn==1.6.1 seaborn==0.13.2

In [ ]:
# Run general imports
import pandas as pd
from IPython.display import display

### Reading iris dataset

For illustrating SVMs in `scikit-learn` we leverage the iris dataset from last week's tutorial.

In [ ]:
from sklearn import datasets

# Load the data
iris = datasets.load_iris()

# We define a Dataframe (tabular structure) with the predictor variables
# and on the other hand a separated vector with the response variable
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target  # Target variable

display(iris_df)

Let's visualize the features of the dataset again:

In [ ]:
import seaborn as sns

# For the purpose of plotting the graphic we make a copy of the data frame
# and add our target column for better visualization
df_label = iris_df.copy()
species_dict = {0: "setosa", 1: "versicolor", 2: "virginica"}
df_label["species"] = pd.Series(y).map(species_dict)

sns.pairplot(
    df_label,
    hue="species",  # Color data points differently based on their value in column  "species"
    plot_kws={"alpha": 0.3},  # Transparency prevents under-representation of overlapping points
)

### Building train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris_df, y, test_size=0.3, random_state=70, stratify=y)

# Python f-strings
print(f"Train dataset size {len(X_train)} / test dataset size {len(X_test)}")

### SVM implementations in scikit-learn

The `scikit-learn` library offers multiple implementations for SVMs (in package `sklearn.svm`) focused on different use-cases and scenarios:

| Class       | Task           | Kernel Support     | Scales Well? |
| ----------- | -------------- | ------------------ | ------------ |
| `SVC`       | Classification | Yes (nonlinear OK) | ❌ Medium     |
| `NuSVC`     | Classification | Yes                | ❌ Medium     |
| `LinearSVC` | Classification | Only Linear        | ✅ Large      |
| `SVR`       | Regression     | Yes                | ❌ Medium     |
| `NuSVR`     | Regression     | Yes                | ❌ Medium     |
| `LinearSVR` | Regression     | Only Linear        | ✅ Large      |

In our tutorial today, we'll focus on the following two implementations:

1. `sklearn.svm.SVC` — Support Vector Classification
- Standard SVM for classification tasks.
- Supports linear, polynomial, RBF, and sigmoid kernels.
- Can use the kernel trick.
- Suitable for small to medium datasets.

2. `sklearn.svm.LinearSVC`: Linear Support Vector Classification
- Optimized for large datasets with a linear kernel only.
- Much faster than `SVC(kernel="linear")`.
- However - does not support the kernel trick.

Please refer to the [documentation page](https://scikit-learn.org/stable/modules/svm.html) for more details.

### SVM training

Let's train a first SVM on the iris dataset

In [ ]:
from sklearn import svm

# Linear SVM classifier
clf_svm = svm.SVC(kernel="linear")
clf_svm.fit(X_train, y_train)
clf_svm

Check the performance of the trained model

In [ ]:
from sklearn.metrics import accuracy_score

# 1. Check the overall accuracy of the model on the validation set
y_pred = clf_svm.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report

# 2. Check the performance per class
report = classification_report(y_test, y_pred, target_names=["setosa", "versicolor", "virginica"])
print(report)

In [ ]:
from sklearn.metrics import confusion_matrix

# 3. Check the confusion matrix to better understand the errors
svm_conf_mat = confusion_matrix(y_test, y_pred)
svm_conf_mat

### Hyperparameters

The `SVC` has three important hyperparameters:

1. `C` (default=`1.0`)
- Controls the trade-off between a smooth decision boundary and classifying training points correctly
- Low C → smoother boundary, more regularization (however might underfit)
- High C → less regularization, fits training data more tightly (risk of overfitting)

2. `kernel` (default=`rbf`)
- Specifies the type of kernel used to transform the data:
- `linear` – linear decision boundary
- `poly` – polynomial kernel
- `rbf` – radial basis function (Gaussian), usually a good default

3. `gamma` (default=`scale`)
- Controls how far the influence of a single training example reaches.
- Only available when using 'rbf', 'poly', 'sigmoid' kernel
- Can be `scale`, `auto` or a fixed number
- `auto`: 1 / n_features
- `scale`: 1 / (n_features * Var(X))
- Low gamma → ‘far’ influence, smoother boundaries.
- High gamma → ‘close’ influence, tighter fit, potential overfitting.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=71, stratify=y_train)

# Python f-strings
print(f"Train dataset size {len(X_train)} / val dataset size {len(X_val)}")

In [ ]:
# Train an SVM classifier with RBF kernel
clf_svm_rbf = svm.SVC(kernel="rbf", C=1.0, gamma=0.1)
clf_svm_rbf.fit(X_train, y_train)
clf_svm_rbf

In [ ]:
# Predict on test set
y_pred = clf_svm_rbf.predict(X_val)
accuracy_score(y_val, y_pred)

**Exercise: Investigate the influence of different kernels and hyper-parameters settings on the performance**

In [ ]:
# Add your code here ...

# 1. Define hyperparameters and instantiate a SVM

# 2. Train a SVM classifier on the training data set

# 3. Validate the performance on the validation set



### Hyperparameter optimization

`sklearn` offers utilities to run hyperparameter optimization using (for example) `GridSearchCV` or `RandomizedSearchCV`.



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameter values we want to explore
parameter_values = {"C": [0.1, 1, 10, 100], "gamma": [0.001, 0.01, 0.1, 1], "kernel": ["linear", "rbf"]}

# Instantiate grid search util
grid_search = GridSearchCV(
    estimator=SVC(),  # Classifier to be optimized
    param_grid=parameter_values,  # Parameter values to explore
    cv=5,  # Number of cross-validations to be performed
    scoring="accuracy",  # Scoring function to rate the different parameter combinations
    n_jobs=-1,  # Number of parallel execution
)

# Run grid search
grid_search.fit(X_train, y_train)

# Show best results
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

In [ ]:
# Use the best classifier to predict target on new instance
y_pred = grid_search.predict(X_val)
accuracy_score(y_val, y_pred)

In [ ]:
# We can also access the best model directly
grid_search.best_estimator_

In [ ]:
# Get the complete result overview of all configurations
results = pd.DataFrame(grid_search.cv_results_)
display(results)

In [ ]:
# Sort the result by test score (rank)
display(results.sort_values("rank_test_score"))

### Visualize the decision boundaries

For illustration purposes, we will reload the data and take only the first two features of the iris data set:

In [ ]:
import matplotlib.pyplot as plt

# Reload data set
iris = datasets.load_iris()
X = pd.DataFrame(iris.data[:, :2], columns=[iris.feature_names[0], iris.feature_names[1]])  # first 2 features
y = iris.target

In [ ]:
display(X)

In [ ]:
# Plot a figure highlight the data points and their classes
plt.figure(figsize=(15, 5))
plt.subplot(121)
plt.scatter(
    x=X[iris.feature_names[0]], y=X[iris.feature_names[1]], c=y, cmap=plt.cm.Set1
)  # Parameter c controls the highlight
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1])
plt.title("Iris dataset")
plt.show()

Re-train an SVM model based only on the two features selected

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

# Train the model using a linear kernel
svc_clf = svm.SVC(kernel="linear", C=1, gamma=1)  # C and gamma are hyper-parameters
svc_clf.fit(X_train, y_train)

Now let's plot the decision boundary for the linear classifier

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay

# Create a figure using matplot lib
fig, ax = plt.subplots()

# Use a utility class from scikit-learn to print the decision boundaries
disp = DecisionBoundaryDisplay.from_estimator(
    svc_clf,
    X,
    response_method="predict",
    cmap=plt.cm.coolwarm,
    alpha=0.8,
    ax=ax,
    xlabel=iris.feature_names[0],
    ylabel=iris.feature_names[1],
)

# Plot the data points
ax.scatter(X[iris.feature_names[0]], X[iris.feature_names[1]], c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
ax.set_xticks(())
ax.set_yticks(())

Now let's compare different models together

In [ ]:
# Define the models that we want to inspect and their configurations
models = [
    svm.SVC(kernel="linear", C=1),
    svm.LinearSVC(C=1, max_iter=10000),
    svm.SVC(kernel="rbf", gamma=0.7, C=1),
    svm.SVC(kernel="poly", degree=3, gamma="auto", C=1),
]

# Fit all the models
models = [clf.fit(X_train, y_train) for clf in models]
models

Now let's compare their decision boundaries graphically

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

# Titles for the plots
titles = (
    "SVC with linear kernel",
    "LinearSVC (linear kernel)",
    "SVC with RBF kernel",
    "SVC with polynomial (degree 3) kernel",
)

# Set-up 2x2 grid for plotting.
fig, sub = plt.subplots(2, 2)
plt.subplots_adjust(wspace=0.4, hspace=0.4)

for clf, title, ax in zip(models, titles, sub.flatten(), strict=False):
    disp = DecisionBoundaryDisplay.from_estimator(
        clf,
        X,
        response_method="predict",
        cmap=plt.cm.coolwarm,
        alpha=0.8,
        ax=ax,
        xlabel=iris.feature_names[0],
        ylabel=iris.feature_names[1],
    )
    ax.scatter(X[iris.feature_names[0]], X[iris.feature_names[1]], c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

**Exercise: Investigate the changes in the decision boundary if changing hyperparameters**

### Implementation hints

**Feature scaling**

The input features of an SVM should always be scaled as SVMs are distance-based models, i.e, they rely heavily on dot products and Euclidean distances (especially when using kernels).

- If features are on different scales, one can dominate the others, leading to poor decision boundaries.
- Unscaled features distort the kernel: The RBF kernel is sensitive to feature magnitude. Large-scale features can skew similarity calculations.
- Optimization becomes harder: Feature imbalance can lead to slow convergence or convergence failure in the underlying solver.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the processing pipeline
pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC(kernel="linear"))])

# Train the classifier using the pipeline
pipeline.fit(X_train, y_train)

In [ ]:
# Nothing changes for the predict method
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

Pipelines can also be used when performing grid search for the best hyperparameters.

In [ ]:
# Define the pipeline
pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])

# For the parameter specification we have to put the name of the component
# before the parameter name (e.g., svc__C for parameter 'C' of component 'svc')
param_grid = {"svc__C": [0.1, 1, 10, 100], "svc__gamma": [0.001, 0.01, 0.1, 1], "svc__kernel": ["rbf"]}

# ... the remainder of the code stays the same
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring="accuracy")
grid.fit(X_train, y_train)

grid.predict(X_test)
accuracy_score(y_test, y_pred)

### Exercise: classify hand writings

In this exercise, we'll use the MNIST dataset,  a classic benchmark dataset in machine learning and computer vision. It contains:

- 70,000 grayscale images of handwritten digits (0 to 9)
- Each image is 8x8 / 28×28 pixels
- Labels correspond to the digit shown in the image

In [ ]:
# Load MNIST dataset (this is actually the 8x8 version; for full 28x28 see below)
from sklearn import datasets

size = 8
digits = datasets.load_digits()
X, y = digits.data, digits.target

# ------------------
# from sklearn.datasets import fetch_openml
# size = 28
# digits = fetch_openml('mnist_784', version=1, as_frame=False)
# X, y = digits.data / 255.0, digits.target.astype(int)

# We only use 2.5k of the points (otherwise training will take too long)
# X = X[:2500]
# y = y[:2500]

In [ ]:
# Let's look at our features
X

In [ ]:
# ... and target
y

The feature values are difficult to interpret. Let's visualize the numbers instead:

In [ ]:
from matplotlib import pyplot as plt

# Number of images to display
num_images = 16
rows, cols = 4, 4  # 4x4 grid

# Create subplots
fig, axes = plt.subplots(rows, cols, figsize=(6, 6))
fig.suptitle("MNIST Digits", fontsize=16)

for i in range(num_images):
    ax = axes[i // cols, i % cols]
    ax.imshow(X[i].reshape(size, size), cmap="gray")
    ax.set_title(f"Label: {y[i]}", fontsize=8)
    ax.axis("off")

plt.tight_layout()
plt.show()

**Now it's your turn:**
- Split the data into a train, validation and test set
- Train a classifier of your choice (e.g. decision tree, logistic regression, SVM)
- Evaluate the performance of the classifier
- Analyze the prediction results

In [ ]:
# You code here ...